In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

# QUICK START: Saudi HR CV Scoring Model
# Copy this entire code into a Google Colab cell and run it
# Make sure you have T4 GPU enabled and uploaded your JSON file

# Step 2: Import libraries
import torch
import json
from datasets import Dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

# Step 3: Load model (try alternatives if this doesn't work)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Alternative: "unsloth/llama-3-8b-instruct-bnb-4bit"
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# Step 4: Setup for fine-tuning
model = FastLanguageModel.get_peft_model(
    model, r=16, target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16, lora_dropout=0, bias="none", use_gradient_checkpointing="unsloth", random_state=3407,
)


# Step 5: Load your dataset
with open('saudi_hr_alpaca_dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Step 6: Format dataset
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def format_prompts(examples):
    texts = []
    for instruction, input_text, output in zip(examples["instruction"], examples["input"], examples["output"]):
        text = alpaca_prompt.format(instruction, input_text, output) + tokenizer.eos_token
        texts.append(text)
    return {"text": texts}

dataset = Dataset.from_list(data).map(format_prompts, batched=True)

# Step 7: Train the model
trainer = SFTTrainer(
    model=model, tokenizer=tokenizer, train_dataset=dataset, dataset_text_field="text",
    max_seq_length=2048, dataset_num_proc=2, packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2, gradient_accumulation_steps=4, warmup_steps=5,
        max_steps=60, learning_rate=2e-4, fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(), logging_steps=1, optim="adamw_8bit",
        weight_decay=0.01, lr_scheduler_type="linear", seed=3407, output_dir="outputs",
        report_to="none", save_strategy="steps", save_steps=20, dataloader_num_workers=0,
    ),
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training completed!")

# Step 8: Test the model
def score_cv(job_posting, cv_details):
    instruction = "Rate this CV against the job posting on a scale of 1-10 and provide a brief explanation for the Saudi HR market context."
    input_text = f"Job Posting: {job_posting}\n\nCV:\n{cv_details}"
    prompt = alpaca_prompt.format(instruction, input_text, "")

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7, do_sample=True, pad_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("### Response:")[-1].strip()

# Step 9: Save model
model.save_pretrained("saudi_hr_model")
tokenizer.save_pretrained("saudi_hr_model")
print("💾 Model saved!")

# Step 10: Test with example
test_job = """Software Engineer - Riyadh Tech Company
Position: Full-time software development
Experience Required: 3+ years
Skills: Python, JavaScript, React, Node.js, Arabic Localization"""

test_cv = """Name: Ahmed Al-Rashid
Nationality: Saudi Arabian
Experience: 4 years software development
Education: Bachelor Computer Science - King Saud University
Skills: Python, JavaScript, React, Node.js, Arabic Localization
Current Role: Software Developer at tech startup"""

result = score_cv(test_job, test_cv)
print("🎯 Test Result:")
print(result)
print("\n🎉 Your Saudi HR CV scoring model is ready to use!")
print("Use: score_cv(job_posting, cv_details) to score any CV")

import random
import re

# Step 9: EVALUATION FUNCTION - Test with 10 samples
def test_model_with_samples():
    """Test the fine-tuned model with 10 random samples from the dataset"""
    print("\n🧪 TESTING MODEL WITH 10 SAMPLES")
    print("=" * 50)

    # Get 10 random samples
    random.seed(42)  # For reproducible results
    samples = random.sample(data, 30)

    correct_predictions = 0
    total_score_diff = 0

    for i, sample in enumerate(samples, 1):
        print(f"\n📋 TEST {i}/10")
        print("-" * 30)

        # Parse the input to extract job posting and CV
        input_parts = sample['input'].split('\n\nCV:\n')
        job_posting = input_parts[0].replace('Job Posting: ', '')
        cv_details = input_parts[1] if len(input_parts) > 1 else ""

        # Get model prediction
        model_prediction = score_cv(job_posting, cv_details)

        # Extract scores from text
        def extract_score(text):
            score_match = re.search(r'Score:\s*(\d+\.?\d*)/10', text)
            return float(score_match.group(1)) if score_match else 0.0

        expected_score = extract_score(sample['output'])
        predicted_score = extract_score(model_prediction)

        # Calculate difference
        score_diff = abs(expected_score - predicted_score)
        total_score_diff += score_diff

        # Check if prediction is good (within 1 point)
        if score_diff <= 1.0:
            correct_predictions += 1
            status = "✅ GOOD"
        else:
            status = "❌ NEEDS WORK"

        # Show results
        print(f"📝 Job: {job_posting[:100]}...")
        print(f"👤 CV: {cv_details[:100]}...")
        print(f"\n✅ Expected: {sample['output']}")
        print(f"🤖 Model Got: {model_prediction}")
        print(f"\n📊 Scores: Expected {expected_score}/10, Got {predicted_score}/10")
        print(f"📈 Difference: {score_diff:.1f} points - {status}")

    # Final results
    accuracy = (correct_predictions / 10) * 100
    avg_diff = total_score_diff / 10

    print(f"\n🎯 FINAL RESULTS")
    print("=" * 50)
    print(f"📊 Accuracy: {accuracy:.0f}% ({correct_predictions}/10 within 1 point)")
    print(f"📈 Average Score Difference: {avg_diff:.2f} points")

    if accuracy >= 80:
        print("🎉 EXCELLENT! Model is performing very well!")
    elif accuracy >= 60:
        print("👍 GOOD! Model is performing reasonably well")
    elif accuracy >= 40:
        print("⚠️ FAIR! Model needs some improvement")
    else:
        print("❌ POOR! Model needs significant improvement")

    return {
        'accuracy': accuracy,
        'avg_difference': avg_diff,
        'correct_predictions': correct_predictions,
        'total_samples': 10
    }

# Step 10: Run the evaluation
evaluation_results = test_model_with_samples()

model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

from google.colab import drive
drive.mount('/content/drive')

# Copy your GGUF model file to that folder
!cp /content/gguf_model/unsloth.Q4_K_M.gguf /content/drive/MyDrive/hrmodel/